In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pickle
import numpy as np
import os
from sklearn import metrics
import importlib
import my_module
importlib.reload(my_module)
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
import codecs
import split
from functools import reduce
    
all_f1= []
all_pre = []
all_recall = []
all_accu = []
for i in range(10):  
    split.split('../../Data/Milingual/emnlp_train.txt','../../Data/Milingual/emnlp_test.txt')
    split.process()

    word2vec = pickle.load(open('../../Data/Milingual/emnlp_embedding.pkl', 'rb'))    

    model = my_module.aspect_model(word2vec, is_bi=True, embedding_trainable=False, loop_num=3)
    model.cuda()
    data = pickle.load(open('../../Data/Milingual/emnlp_train.pkl', 'rb'))
    test = pickle.load(open('../../Data/Milingual/emnlp_test.pkl', 'rb'))
    trainable_param = list(model.parameters())
    trainable_param = filter(lambda x: x.requires_grad, trainable_param)
    opt = optim.Adam(trainable_param, lr=0.001)

    crossentropy = nn.CrossEntropyLoss()
    epochs = 10
    max_f1 = 0


    for epoch in range(epochs):
        for i, batch_data in enumerate(data):
            model.train(mode=True)
            opt.zero_grad()
            text, aspect, position, label, _, _ = zip(*batch_data)
            text, aspect, position,label = np.array(text), aspect, np.array(position), np.array(label)
            batch_size = text.shape[0]
            _text = Variable(torch.LongTensor(text).cuda())
            _pos = Variable(torch.FloatTensor(position).cuda())
            _label = Variable(torch.LongTensor(np.argmax(label, 1)).cuda())
            hidden = model.init_hidden(batch_size)
            logits, probs = model(_text, aspect, _pos, hidden)
            loss = crossentropy(logits, _label)
            loss.backward()
            opt.step()
            i += 1
            if i%1 == 0:
                preds = []
                pred_probs = []
                test_label = []
                inxs = []
                clause_inxs = []
                for batch_data in test:
                    model.eval()
                    text, aspect, position, label, inx, clause_inx = zip(*batch_data)
                    text, aspect, position, label = np.array(text), aspect, np.array(position), np.array(label)
                    batch_size = text.shape[0]
                    _text = Variable(torch.LongTensor(text).cuda())
                    _pos = Variable(torch.FloatTensor(position).cuda())
                    hidden = model.init_hidden(batch_size)
                    _, probs = model(_text, aspect, _pos, hidden)
                    pred = list(np.argmax(probs.data.cpu().numpy(), 1))
                    preds.extend(pred)
                    test_label.extend(list(np.argmax(label, 1)))
                    inxs.extend(inx)
                    clause_inxs.extend(clause_inx)
                    pred_probs.extend(probs.data.cpu().numpy()[:, 1])
                acc = metrics.accuracy_score(np.array(test_label), np.array(preds))
                pre = metrics.precision_score(np.array(test_label), np.array(preds))#, average='macro'
                rec = metrics.recall_score(np.array(test_label), np.array(preds))
                f1 = metrics.f1_score(np.array(test_label), np.array(preds))
                if f1 > max_f1:
                    max_f1 = f1
                    max_acc = acc
                    max_pre = pre
                    max_rec = rec
                    max_epoch = epoch
                    file_name = 'emnlp_f1_'+str(f1) + '_pre_'+str(pre)+'_rec_'+str(rec)+'_epoch_'+str(epoch)
                    with open('../../Data/Prediction/' + file_name, 'w') as f_w:
                        for inx_i in range(len(inxs)):
                            f_w.write(inxs[inx_i] + ' ' + clause_inxs[inx_i] + ' ' + str(pred_probs[inx_i]) + '\n')
                print ('epoch: %d, accuracy: %f, precision: %f, recall: %f, f1: %f, loss:%f'%(epoch, acc, pre, rec, f1, loss.data[0]))
    print ('---max---epoch: %d, accuracy: %f, precision: %f, recall: %f, f1: %f---max---'%(max_epoch, max_acc, max_pre, max_rec, max_f1))
    all_f1.append(max_f1)
    all_pre.append(max_pre)
    all_recall.append(max_rec)
    all_accu.append(max_acc)
print(all_f1)
print(all_pre)
print(all_recall)
print(all_accu)
f1 = reduce(lambda x,y:x+y,all_f1)/10
recall = reduce(lambda x,y:x+y,all_recall)/10
precision = reduce(lambda x,y:x+y,all_pre)/10
accu = reduce(lambda x,y:x+y,all_accu)/10
print(f1,recall,precision,accu)

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


2105


Loading model cost 0.548 seconds.
Prefix dict has been built succesfully.


26217 1952
epoch: 0, accuracy: 0.512187, precision: 0.089142, recall: 0.618605, f1: 0.155829, loss:0.655681
epoch: 0, accuracy: 0.563643, precision: 0.093182, recall: 0.572093, f1: 0.160261, loss:1.397756
epoch: 0, accuracy: 0.630670, precision: 0.089888, recall: 0.446512, f1: 0.149649, loss:1.622150
epoch: 0, accuracy: 0.665538, precision: 0.075741, recall: 0.320930, f1: 0.122558, loss:0.603240
epoch: 0, accuracy: 0.657414, precision: 0.084463, recall: 0.376744, f1: 0.137990, loss:1.323836
epoch: 0, accuracy: 0.631347, precision: 0.088512, recall: 0.437209, f1: 0.147220, loss:1.273041
epoch: 0, accuracy: 0.612729, precision: 0.067907, recall: 0.339535, f1: 0.113178, loss:2.083736
epoch: 0, accuracy: 0.564997, precision: 0.057851, recall: 0.325581, f1: 0.098246, loss:0.745911
epoch: 0, accuracy: 0.517603, precision: 0.053835, recall: 0.339535, f1: 0.092934, loss:0.651424
epoch: 0, accuracy: 0.468856, precision: 0.055774, recall: 0.395349, f1: 0.097757, loss:0.968341
epoch: 0, accuracy:

epoch: 0, accuracy: 0.849018, precision: 0.309720, recall: 0.874419, f1: 0.457421, loss:0.370092
epoch: 0, accuracy: 0.845295, precision: 0.304839, recall: 0.879070, f1: 0.452695, loss:0.412146
epoch: 0, accuracy: 0.835139, precision: 0.292049, recall: 0.888372, f1: 0.439586, loss:0.359131
epoch: 0, accuracy: 0.818551, precision: 0.272984, recall: 0.897674, f1: 0.418655, loss:0.419052
epoch: 0, accuracy: 0.811781, precision: 0.266758, recall: 0.906977, f1: 0.412262, loss:0.503351
epoch: 0, accuracy: 0.804333, precision: 0.260238, recall: 0.916279, f1: 0.405350, loss:0.198766
epoch: 0, accuracy: 0.819905, precision: 0.276446, recall: 0.911628, f1: 0.424242, loss:0.779591
epoch: 0, accuracy: 0.836493, precision: 0.293846, recall: 0.888372, f1: 0.441618, loss:0.486950
epoch: 0, accuracy: 0.853758, precision: 0.315789, recall: 0.865116, f1: 0.462687, loss:0.446015
epoch: 0, accuracy: 0.885579, precision: 0.369427, recall: 0.809302, f1: 0.507289, loss:0.541114
epoch: 0, accuracy: 0.903182, 

...

epoch: 1, accuracy: 0.892011, precision: 0.388412, recall: 0.841860, f1: 0.531571, loss:0.064611
epoch: 1, accuracy: 0.890995, precision: 0.385439, recall: 0.837209, f1: 0.527859, loss:0.226109
epoch: 1, accuracy: 0.893026, precision: 0.390929, recall: 0.841860, f1: 0.533923, loss:0.266246
epoch: 1, accuracy: 0.892688, precision: 0.390086, recall: 0.841860, f1: 0.533137, loss:0.047513
epoch: 1, accuracy: 0.892011, precision: 0.389362, recall: 0.851163, f1: 0.534307, loss:0.055722
epoch: 1, accuracy: 0.890995, precision: 0.387368, recall: 0.855814, f1: 0.533333, loss:0.067984
epoch: 1, accuracy: 0.889641, precision: 0.384134, recall: 0.855814, f1: 0.530259, loss:0.216780
epoch: 1, accuracy: 0.889641, precision: 0.384134, recall: 0.855814, f1: 0.530259, loss:0.037025
epoch: 1, accuracy: 0.892688, precision: 0.391026, recall: 0.851163, f1: 0.535871, loss:0.296805
epoch: 1, accuracy: 0.896073, precision: 0.398230, recall: 0.837209, f1: 0.539730, loss:0.300055
epoch: 1, accuracy: 0.898104, 

epoch: 1, accuracy: 0.890995, precision: 0.389691, recall: 0.879070, f1: 0.540000, loss:0.084446
epoch: 1, accuracy: 0.891334, precision: 0.390496, recall: 0.879070, f1: 0.540773, loss:0.241262
epoch: 1, accuracy: 0.891672, precision: 0.390852, recall: 0.874419, f1: 0.540230, loss:0.102495
epoch: 1, accuracy: 0.892011, precision: 0.391667, recall: 0.874419, f1: 0.541007, loss:0.386609
epoch: 1, accuracy: 0.893026, precision: 0.394130, recall: 0.874419, f1: 0.543353, loss:0.190913
epoch: 1, accuracy: 0.895396, precision: 0.399573, recall: 0.869767, f1: 0.547584, loss:0.311545
epoch: 1, accuracy: 0.896073, precision: 0.400862, recall: 0.865116, f1: 0.547865, loss:0.092873
epoch: 1, accuracy: 0.896412, precision: 0.401728, recall: 0.865116, f1: 0.548673, loss:0.080090
epoch: 1, accuracy: 0.896412, precision: 0.401728, recall: 0.865116, f1: 0.548673, loss:0.061159
epoch: 1, accuracy: 0.896412, precision: 0.401728, recall: 0.865116, f1: 0.548673, loss:0.201432
epoch: 1, accuracy: 0.895735, 

epoch: 1, accuracy: 0.877116, precision: 0.355469, recall: 0.846512, f1: 0.500688, loss:0.067342
epoch: 1, accuracy: 0.878131, precision: 0.359223, recall: 0.860465, f1: 0.506849, loss:0.106592
epoch: 1, accuracy: 0.879147, precision: 0.361868, recall: 0.865116, f1: 0.510288, loss:0.171577
epoch: 1, accuracy: 0.882532, precision: 0.368000, recall: 0.855814, f1: 0.514685, loss:0.328238
epoch: 1, accuracy: 0.883209, precision: 0.368952, recall: 0.851163, f1: 0.514768, loss:0.286124
epoch: 1, accuracy: 0.884902, precision: 0.372188, recall: 0.846512, f1: 0.517045, loss:0.259296
epoch: 1, accuracy: 0.888287, precision: 0.379958, recall: 0.846512, f1: 0.524496, loss:0.292877
epoch: 1, accuracy: 0.892349, precision: 0.388286, recall: 0.832558, f1: 0.529586, loss:0.309130
epoch: 1, accuracy: 0.896750, precision: 0.399554, recall: 0.832558, f1: 0.539970, loss:0.395499
epoch: 1, accuracy: 0.898781, precision: 0.404110, recall: 0.823256, f1: 0.542113, loss:0.150668
epoch: 1, accuracy: 0.899458, 

epoch: 1, accuracy: 0.882871, precision: 0.369781, recall: 0.865116, f1: 0.518106, loss:0.280668
epoch: 1, accuracy: 0.882871, precision: 0.369781, recall: 0.865116, f1: 0.518106, loss:0.155349
epoch: 1, accuracy: 0.882532, precision: 0.368526, recall: 0.860465, f1: 0.516039, loss:0.122840
epoch: 1, accuracy: 0.883886, precision: 0.371486, recall: 0.860465, f1: 0.518934, loss:0.130047
epoch: 1, accuracy: 0.885240, precision: 0.373984, recall: 0.855814, f1: 0.520509, loss:0.230326
epoch: 1, accuracy: 0.887949, precision: 0.380165, recall: 0.855814, f1: 0.526466, loss:0.064598
epoch: 1, accuracy: 0.889980, precision: 0.384937, recall: 0.855814, f1: 0.531025, loss:0.172406
epoch: 1, accuracy: 0.892011, precision: 0.389831, recall: 0.855814, f1: 0.535662, loss:0.098695
epoch: 1, accuracy: 0.894719, precision: 0.396552, recall: 0.855814, f1: 0.541973, loss:0.190817
epoch: 1, accuracy: 0.897089, precision: 0.402198, recall: 0.851163, f1: 0.546269, loss:0.140109
epoch: 1, accuracy: 0.894719, 

epoch: 2, accuracy: 0.885240, precision: 0.373469, recall: 0.851163, f1: 0.519149, loss:0.049293
epoch: 2, accuracy: 0.887271, precision: 0.378099, recall: 0.851163, f1: 0.523605, loss:0.078640
epoch: 2, accuracy: 0.889303, precision: 0.383333, recall: 0.855814, f1: 0.529496, loss:0.108535
epoch: 2, accuracy: 0.890657, precision: 0.386555, recall: 0.855814, f1: 0.532562, loss:0.127105
epoch: 2, accuracy: 0.892349, precision: 0.390192, recall: 0.851163, f1: 0.535088, loss:0.071797
epoch: 2, accuracy: 0.894042, precision: 0.393939, recall: 0.846512, f1: 0.537666, loss:0.098880
epoch: 2, accuracy: 0.895058, precision: 0.396061, recall: 0.841860, f1: 0.538690, loss:0.187645
epoch: 2, accuracy: 0.897766, precision: 0.403118, recall: 0.841860, f1: 0.545181, loss:0.190181
epoch: 2, accuracy: 0.896750, precision: 0.400442, recall: 0.841860, f1: 0.542729, loss:0.120715
epoch: 2, accuracy: 0.896073, precision: 0.399123, recall: 0.846512, f1: 0.542474, loss:0.097698
epoch: 2, accuracy: 0.895735, 